In [ ]:
#%pip install -U git+https://github.com/sybrenstuvel/flickrapi.git@master

In [1]:
from settings import *
from flickrapi import FlickrAPI
import time
import os
import requests
import math

In [2]:
def download_photos(emotion, max_download=500):
    flickr = FlickrAPI(FL_API_KEY, FL_API_SECRET, format='parsed-json')
    pages = math.ceil(max_download/500)
    search_query = emotion + " dog"
    photos = []
    
    save_path = os.path.join(IMAGE_DIR, emotion)
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        
    for p in range(0, pages):
        photo_data = flickr.photos.search(text=search_query,
                             extras='url_l',
                             per_page=max_download,
                             sort='relevance',
                             safe_search=1,
                             page=p+1
                            )
        photos += photo_data["photos"]["photo"]
        time.sleep(1.1)
    
    
    urls = []
    for photo in photos:
        try:
            url = photo.get('url_l')
            if url is None:
                url = f"https://farm{photo.get('farm')}.staticflickr.com/{photo.get('server')}/{photo.get('id')}_{photo.get('secret')}_b.jpg"

            photo_name = os.path.basename(url)
            
            extension = photo_name.split(".")[-1]
            if extension not in VALID_EXTENSIONS:
                continue
            
            photo_path = os.path.join(save_path, photo_name)
            if os.path.exists(photo_path):
                continue
                
            photo_data = requests.get(url, timeout=10).content
            
            with open(photo_path, 'wb+') as f:
                f.write(photo_data)

            urls.append(url)
            time.sleep(1.01)
        except Exception:
            pass

In [ ]:
for emotion in FOLDERS:
    download_photos(emotion, 5000)